In [2]:
import dask
dask.config.set({"dataframe.query-planning": False})


import numpy as np
from pathlib import Path
import json
import pandas as pd
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import sys
sys.path.extend(['../../scripts','../../scripts/xenium'])
import readwrite
cfg = readwrite.config()

## Load data

In [3]:
# cfg paths
xenium_dir = Path(cfg['xenium_processed_data_dir'])
xenium_std_seurat_analysis_dir = Path(cfg['xenium_std_seurat_analysis_dir'])
xenium_cell_type_annotation_dir = Path(cfg['xenium_cell_type_annotation_dir'])
results_dir = Path(cfg['results_dir'])
palette_dir = Path(cfg['xenium_metadata_dir'])

# Params
# probably only need to run for lognorm data
normalisations = ['lognorm',]
layers = ['data',]
references = ['matched_reference_combo']
methods = ['rctd_class_aware']
levels = ['Level2']

n_neighbors = 10
n_permutations = 30
n_repeats = 5
top_n = 20
top_n_lr = 10
scoring = 'f1'
markers = 'diffexpr' #'/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/markers/cellmarker_cell_types_markers.json'

# needed to get unique cell types names for each level
# cell_types_palette = pd.read_csv(palette_dir / 'col_palette_cell_types_combo.csv')

df_diffexpr = {}
df_markers_rank_significance_diffexpr = {}
for segmentation in (segmentations := xenium_std_seurat_analysis_dir.iterdir()):
    if segmentation.stem in ['proseg_mode','proseg_expected']:
        continue
    for condition in (conditions := segmentation.iterdir()): 
        for panel in (panels := condition.iterdir()):
            for donor in (donors := panel.iterdir()):
                for sample in (samples := donor.iterdir()):
                    for normalisation in normalisations:
                        for layer in layers:
                            # for reference in references:
                            #     for method in methods:
                            #         for level in levels:

                                        k = (segmentation.stem,condition.stem,panel.stem,donor.stem,sample.stem)
                                        name = '/'.join(k)

                                        out_file_df_diffexpr = results_dir / f'contamination_metrics/{name}/{normalisation}/{layer}_diffexpr.parquet'
                                        out_file_df_markers_rank_significance_diffexpr = results_dir / f'contamination_metrics/{name}/{normalisation}/{layer}_markers_rank_significance_diffexpr.json'

                                        df_diffexpr[k] = pd.read_parquet(out_file_df_diffexpr)
                                        df_markers_rank_significance_diffexpr[k] = pd.read_parquet(out_file_df_markers_rank_significance_diffexpr)

In [ ]:
df_markers_rank_significance_diffexpr[k]#.groupby['hypergeometric_pvalue']

MultiIndex([(   'endothelial cell', 'malignant cell', 0),
            (         'fibroblast', 'malignant cell', 0),
            (    'epithelial cell', 'malignant cell', 0),
            ( 'cycling lymphocyte', 'malignant cell', 0),
            (     'dendritic cell', 'malignant cell', 0),
            (        'plasma cell', 'malignant cell', 0),
            (           'pericyte', 'malignant cell', 0),
            (             'T cell', 'malignant cell', 0),
            (           'monocyte', 'malignant cell', 0),
            ( 'smooth muscle cell', 'malignant cell', 0),
            ...
            (     'dendritic cell',         'B cell', 0),
            (        'plasma cell',         'B cell', 0),
            (           'pericyte',         'B cell', 0),
            (             'T cell',         'B cell', 0),
            (           'monocyte',         'B cell', 0),
            ( 'smooth muscle cell',         'B cell', 0),
            ('natural killer cell',         'B cell', 0)